In [2]:
from datasets import load_dataset

ds = load_dataset("iamtarun/code_instructions_120k_alpaca")

KeyboardInterrupt: 

In [ ]:
# Check keys (splits)
print(ds)


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt'],
        num_rows: 121959
    })
})


In [ ]:
import json

# Perform 70-30 train-test split
split_dataset = ds['train'].train_test_split(test_size=0.3, seed=42)

# Save train data to JSONL
with open("train.jsonl", "w", encoding="utf-8") as f_train:
    for example in split_dataset['train']:
        json.dump({
            "prompt": example["prompt"],
            "output": example["output"]
        }, f_train)
        f_train.write('\n')

# Save test data to JSONL
with open("test.jsonl", "w", encoding="utf-8") as f_test:
    for example in split_dataset['test']:
        json.dump({
            "prompt": example["prompt"],
            "output": example["output"]
        }, f_test)
        f_test.write('\n')


In [ ]:
# Main execution
if __name__ == "__main__":
    # Translate the main dataset - RESUME FROM LINE 16751
    input_file = "cleaned_test.jsonl"
    output_file = "final_test.jsonl"
    start_index = 16750  # Starting from index 16750 (line 16751)
    
    print(f"🚀 Starting translation from line {start_index + 1}")
    
    # Start translation with resume capability
    translated_data, failed_data = translate_dataset(input_file, output_file, start_from=start_index)
    
    # Retry failed translations if any
    if failed_data:
        print("\nRetrying failed translations...")
        retry_failed_translations("translation_failures.jsonl", "retry_success.jsonl")
    
    # Check translation quality
    if os.path.exists(output_file):
        check_translation_quality(output_file)
    
    print("\n🎉 Translation process completed!")

Loading dataset...
Loaded 36588 entries
Starting translation process...


Translating:   0%|          | 50/36588 [00:38<7:54:16,  1.28it/s] 

Processed 50/36588 items


Translating:   0%|          | 100/36588 [01:18<6:39:13,  1.52it/s]

Processed 100/36588 items


...

Translating:  46%|████▌     | 16650/36588 [6:06:20<8:23:56,  1.52s/it]

Processed 16650/36588 items


Translating:  46%|████▌     | 16699/36588 [6:07:29<7:07:58,  1.29s/it] 

Processed 16700/36588 items


Translating:  46%|████▌     | 16749/36588 [6:08:41<8:54:06,  1.62s/it]

In [ ]:
import json
import time
import re
from deep_translator import GoogleTranslator
from tqdm import tqdm

# === Load failed long translations ===
with open("translation_failures.jsonl", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f if line.strip()]

success = []
failed = []

def chunk_text(text, max_len=4500):
    # Split into sentences using regex (handles '.', '?', '!')
    sentences = re.split(r'(?<=[.?!])\s+', text)
    chunks = []
    current = ""
    for s in sentences:
        if len(current) + len(s) < max_len:
            current += s + " "
        else:
            chunks.append(current.strip())
            current = s + " "
    if current:
        chunks.append(current.strip())
    return chunks

for item in tqdm(data):
    text = item["instruction"]
    if len(text) < 4500:
        failed.append(item)
        continue

    try:
        chunks = chunk_text(text)
        translated_chunks = []
        for chunk in chunks:
            translated = GoogleTranslator(source='en', target='ms').translate(chunk)
            translated_chunks.append(translated)
            time.sleep(1)  # Avoid rate-limiting

        full_translation = " ".join(translated_chunks)
        item["text_ms"] = full_translation
        success.append(item)
    except Exception as e:
        print(f"Failed: {e}")
        item["text_ms"] = "[RETRANSLATION_FAILED]"
        failed.append(item)

# === Save retried translations ===
with open("retranslated_longtext_fixed.jsonl", "w", encoding="utf-8") as f:
    for item in success:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

with open("still_failed_longtext.jsonl", "w", encoding="utf-8") as f:
    for item in failed:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"✅ Retranslated: {len(success)}")
print(f"❌ Still failed: {len(failed)}")


100%|██████████| 4/4 [00:20<00:00,  5.05s/it]

Failed: Use this code to remove \n and \t from island_list and commodity_list respectively and print the updated lists:

```
island_list = ['Armstrong Island', 'Atchafalaya Island', 'Immokalee Island', 'Moultrie Island', 'Sho-ke Island', 'Sirius Island',
               'Tumult Island', 'The Beaufort Islands', "Messier's Crown", 'Nunataq Island', 'Paollu Island', 'Qaniit Island',
               'Ancoraggio Island', 'Fluke Island', 'Kakraphoon Island', 'Eagle Archipelago', 'Cambium Island', "Hubble's Eye",
               'Ilha da Aguia', 'Ix Chel', 'Manu Island', 'Admiral Island', 'Basset Island', 'Bryher Island', 'Cromwell Island',
               'Hook Shelf', 'Isle of Kent', 'Lincoln Island', 'Wensleydale', 'Anegada Island', 'Barnard Island', 'The Lowland Hundred',
               'Lyonesse Island', 'Myvatn Island', 'Arakoua Island', 'Aten Island', 'Barbary Island', 'Caravanserai Island',
               'Kasidim Island', 'Kiwara Island', 'Terjit Island', 'Tichka Plateau', 'Aimuari Islan

In [ ]:
import json
import re

# Step 1: Load original data with `prompt`
with open("original train.jsonl", "r", encoding="utf-8") as f:
    original_data = [json.loads(line.strip()) for line in f if line.strip()]

# Step 2: Extract input text from prompt
extracted_inputs = []
for item in original_data:
    prompt = item.get("prompt", "")
    input_match = re.search(r"### Input:\s*(.*?)\s*### Response:", prompt, re.DOTALL)
    if input_match:
        extracted_inputs.append(input_match.group(1).strip())
    else:
        extracted_inputs.append("")  # If input not found, use empty string

# Step 3: Load the final cleaned dataset (without input)
with open("final_train.jsonl", "r", encoding="utf-8") as f:
    final_data = [json.loads(line.strip()) for line in f if line.strip()]

# Step 4: Add input after instruction without changing other fields
updated_data = []
for entry, input_text in zip(final_data, extracted_inputs):
    new_entry = {}

    # Ensure 'instruction' is first
    new_entry["instruction"] = entry.get("instruction", "")
    # Add extracted input
    new_entry["input"] = input_text
    # Add the rest of the keys (order: output, text_ms, etc.)
    for key, value in entry.items():
        if key not in new_entry:
            new_entry[key] = value

    updated_data.append(new_entry)

# Step 5: Save to a new file
with open("final_train_with_input.jsonl", "w", encoding="utf-8") as f:
    for entry in updated_data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")


In [ ]:
import json

# Read the source file (English with outputs)
with open("cleaned_train.jsonl", "r", encoding="utf-8") as f:
    english_data = [json.loads(line.strip()) for line in f if line.strip()]

# Read the target file (Malay with empty outputs)
with open("Python_Train_Data.jsonl", "r", encoding="utf-8") as f:
    malay_data = [json.loads(line.strip()) for line in f if line.strip()]

# Copy outputs from English to Malay data
for i in range(min(len(english_data), len(malay_data))):
    malay_data[i]["output"] = english_data[i]["output"]

# Write the updated data back to Python_Train_Data.jsonl
with open("Python_Train_Data.jsonl", "w", encoding="utf-8") as f:
    for item in malay_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"✅ Copied outputs from {len(english_data)} entries")

✅ Copied outputs from 85371 entries


In [ ]:
import json
import time
import re
from deep_translator import GoogleTranslator
from tqdm import tqdm
import os

# Load your cleaned_test.jsonl dataset
def load_dataset(file_path):
    """Load JSONL dataset"""
    data = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line.strip()))
    return data

# Text chunking function for long instructions
def chunk_text(text, max_len=4500):
    """Split text into chunks to avoid API limits"""
    # Split into sentences using regex
    sentences = re.split(r'(?<=[.?!])\s+', text)
    chunks = []
    current = ""
    
    for sentence in sentences:
        if len(current) + len(sentence) < max_len:
            current += sentence + " "
        else:
            if current:
                chunks.append(current.strip())
            current = sentence + " "
    
    if current:
        chunks.append(current.strip())
    
    return chunks

# Translation function with retry mechanism
def translate_text(text, source='en', target='ms', max_retries=3):
    """Translate text with retry mechanism"""
    for attempt in range(max_retries):
        try:
            if len(text) <= 4500:
                # Direct translation for short text
                translator = GoogleTranslator(source=source, target=target)
                return translator.translate(text)
            else:
                # Chunked translation for long text
                chunks = chunk_text(text)
                translated_chunks = []
                
                for chunk in chunks:
                    translator = GoogleTranslator(source=source, target=target)
                    translated_chunk = translator.translate(chunk)
                    translated_chunks.append(translated_chunk)
                    time.sleep(0.5)  # Rate limiting
                
                return " ".join(translated_chunks)
        
        except Exception as e:
            print(f"Translation attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                time.sleep(2)  # Wait before retry
            else:
                return f"[TRANSLATION_FAILED: {str(e)}]"

# Main translation process with resume capability
def translate_dataset(input_file, output_file, start_from=0):
    """Translate the instruction field of the dataset"""
    # Load dataset
    print("Loading dataset...")
    data = load_dataset(input_file)
    print(f"Loaded {len(data)} entries")
    
    # Skip to start_from if resuming
    if start_from > 0:
        print(f"🔄 Resuming from entry {start_from}")
        data = data[start_from:]
        print(f"Processing remaining {len(data)} entries")
    
    translated_data = []
    failed_translations = []
    
    # Load existing progress if resuming
    if start_from > 0 and os.path.exists(f"temp_{output_file}"):
        print("📂 Loading existing progress...")
        try:
            existing_data = load_dataset(f"temp_{output_file}")
            translated_data = existing_data
            print(f"Loaded {len(existing_data)} previously translated entries")
        except:
            print("⚠️ Could not load existing progress, starting fresh")
    
    print("Starting translation process...")
    for i, item in enumerate(tqdm(data, desc="Translating")):
        actual_index = i + start_from  # Track actual position in original dataset
        try:
            # Create new item with translated instruction
            new_item = item.copy()
            
            # Translate the instruction field
            original_instruction = item.get("instruction", "")
            if original_instruction:
                translated_instruction = translate_text(original_instruction)
                new_item["instruction"] = translated_instruction
                new_item["instruction_en"] = original_instruction  # Keep original
            
            translated_data.append(new_item)
            
            # Rate limiting - sleep between requests
            time.sleep(0.5)
            
            # Save progress every 50 items
            if (i + 1) % 50 == 0:
                print(f"Processed {actual_index + 1}/{start_from + len(data)} items (Current batch: {i + 1}/{len(data)})")
                # Save intermediate results
                save_intermediate_results(translated_data, f"temp_{output_file}")
        
        except Exception as e:
            print(f"Error processing item {actual_index}: {e}")
            item["translation_error"] = str(e)
            item["original_index"] = actual_index  # Track original position
            failed_translations.append(item)
    
    # Save final results
    save_results(translated_data, output_file)
    
    # Save failed translations for retry
    if failed_translations:
        save_results(failed_translations, "translation_failures.jsonl")
    
    print(f"✅ Successfully translated: {len(translated_data)}")
    print(f"❌ Failed translations: {len(failed_translations)}")
    
    return translated_data, failed_translations

def save_results(data, filename):
    """Save results to JSONL file"""
    with open(filename, "w", encoding="utf-8") as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")
    print(f"Saved {len(data)} items to {filename}")

def save_intermediate_results(data, filename):
    """Save intermediate results during processing"""
    with open(filename, "w", encoding="utf-8") as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

# Retry failed translations
def retry_failed_translations(failed_file, output_file):
    """Retry previously failed translations"""
    print("Retrying failed translations...")
    
    # Load failed translations
    failed_data = load_dataset(failed_file)
    
    success = []
    still_failed = []
    
    for item in tqdm(failed_data, desc="Retrying"):
        try:
            # Remove previous error info
            if "translation_error" in item:
                del item["translation_error"]
            
            # Try translation again
            original_instruction = item.get("instruction", "")
            if original_instruction:
                translated_instruction = translate_text(original_instruction)
                
                if not translated_instruction.startswith("[TRANSLATION_FAILED"):
                    item["instruction"] = translated_instruction
                    item["instruction_en"] = original_instruction
                    success.append(item)
                else:
                    item["translation_error"] = "Retry failed"
                    still_failed.append(item)
            
            time.sleep(1)  # Longer delay for retries
        
        except Exception as e:
            item["translation_error"] = str(e)
            still_failed.append(item)
    
    # Save results
    if success:
        save_results(success, output_file)
    if still_failed:
        save_results(still_failed, "still_failed_translations.jsonl")
    
    print(f"✅ Retry successful: {len(success)}")
    print(f"❌ Still failed: {len(still_failed)}")

# Quality check function
def check_translation_quality(translated_file, sample_size=10):
    """Check a sample of translations for quality"""
    data = load_dataset(translated_file)
    
    print(f"\n=== Translation Quality Check ===")
    print(f"Checking {min(sample_size, len(data))} random samples:\n")
    
    import random
    sample_indices = random.sample(range(len(data)), min(sample_size, len(data)))
    
    for i, idx in enumerate(sample_indices):
        item = data[idx]
        print(f"Sample {i+1}:")
        print(f"English: {item.get('instruction_en', 'N/A')[:100]}...")
        print(f"Malay:   {item.get('instruction', 'N/A')[:100]}...")
        print("-" * 80)

# Main execution to resume from line 16751
if __name__ == "__main__":
    # Configuration
    input_file = "cleaned_test.jsonl"
    output_file = "final_test.jsonl"
    start_index = 36550  # Starting from index 16750 (line 16751)
    
    print(f"🚀 Starting translation from line {start_index + 1}")
    
    # Start translation with resume capability
    translated_data, failed_data = translate_dataset(input_file, output_file, start_from=start_index)
    
    # Retry failed translations if any
    if failed_data:
        print("\nRetrying failed translations...")
        retry_failed_translations("translation_failures.jsonl", "retry_success.jsonl")
    
    # Check translation quality
    if os.path.exists(output_file):
        check_translation_quality(output_file)
    
    print("\n🎉 Translation process completed!")

🚀 Starting translation from line 36572
Loading dataset...
Loaded 36588 entries
🔄 Resuming from entry 36571
Processing remaining 17 entries
📂 Loading existing progress...
Loaded 36571 previously translated entries
Starting translation process...


Translating:   0%|          | 0/17 [00:00<?, ?it/s]

Processed 36572/36588 items (Current batch: 1/17)


Translating:   6%|▌         | 1/17 [00:01<00:22,  1.38s/it]

Processed 36573/36588 items (Current batch: 2/17)


...

Processed 36587/36588 items (Current batch: 16/17)


Translating:  94%|█████████▍| 16/17 [00:24<00:01,  1.35s/it]

Processed 36588/36588 items (Current batch: 17/17)


Translating: 100%|██████████| 17/17 [00:25<00:00,  1.52s/it]


Saved 36588 items to final_test.jsonl
✅ Successfully translated: 36588
❌ Failed translations: 0

=== Translation Quality Check ===
Checking 10 random samples:

Sample 1:
English: Generate a Python program that takes in a string of text and returns the count of each unique word i...
Malay:   Menjana program python yang mengambil rentetan teks dan mengembalikan kiraan setiap perkataan yang u...
--------------------------------------------------------------------------------
Sample 2:
English: Construct a program in Kotlin that swaps the positions of two elements in an array....
Malay:   Membina program di Kotlin yang menukar kedudukan dua elemen dalam array....
--------------------------------------------------------------------------------
Sample 3:
English: Create a PHP REST API to manage books in a library database....
Malay:   Buat API PHP REST untuk menguruskan buku dalam pangkalan data perpustakaan....
--------------------------------------------------------------------------------

In [13]:
def copy_jsonl_with_line_control(source_file, target_file, start_line=1, end_line=None):
    """
    Copy JSONL entries with line control and validation
    """
    import json
    
    copied_count = 0
    error_count = 0
    
    try:
        with open(source_file, "r", encoding="utf-8") as src, \
             open(target_file, "w", encoding="utf-8") as target:
            
            for line_num, line in enumerate(src, 1):
                # Check if we're in the desired range
                if line_num < start_line:
                    continue
                if end_line and line_num > end_line:
                    break
                
                # Validate and copy JSON line
                if line.strip():
                    try:
                        # Validate JSON
                        json.loads(line.strip())
                        target.write(line)
                        copied_count += 1
                    except json.JSONDecodeError:
                        print(f"⚠️ Invalid JSON at line {line_num}: {line[:50]}...")
                        error_count += 1
        
        print(f"✅ Copied {copied_count} valid JSON entries")
        print(f"❌ Skipped {error_count} invalid entries")
        
    except Exception as e:
        print(f"❌ Error: {e}")

# Usage examples:
if __name__ == "__main__":
    
    # Copy remaining untranslated data
    copy_jsonl_with_line_control("final_train.jsonl", "final_train3.jsonl", start_line=68001)
    
    # Copy first batch for testing
    #copy_jsonl_with_line_control("final_train.jsonl", "final_train2.jsonl",start_line=28953, end_line=68000)

✅ Copied 17371 valid JSON entries
❌ Skipped 0 invalid entries


In [5]:
import json

def export_long_instructions(file_path, output_file, min_length=500):
    """
    Export entries with long instructions to a separate file
    """
    long_entries = []
    
    with open(file_path, "r", encoding="utf-8") as f:
        for line_num, line in enumerate(f, 1):
            if line.strip():
                try:
                    data = json.loads(line.strip())
                    instruction = data.get("instruction", "")
                    
                    if len(instruction) > min_length:
                        data["original_line_number"] = line_num
                        data["instruction_length"] = len(instruction)
                        long_entries.append(data)
                
                except json.JSONDecodeError:
                    continue
    
    # Save to file
    with open(output_file, "w", encoding="utf-8") as f:
        for entry in long_entries:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")
    
    print(f"✅ Exported {len(long_entries)} long instructions to {output_file}")
    return long_entries

# Usage
if __name__ == "__main__":
    # Export all instructions > 500 chars
    export_long_instructions("final_test.jsonl", "long_instructions_malay.jsonl", min_length=500)

✅ Exported 137 long instructions to long_instructions_malay.jsonl


In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load a multilingual model (e.g., LaBSE for many languages including English and Malay)
# You might need to install: pip install sentence-transformers
model = SentenceTransformer("sentence-transformers/LaBSE")

def get_semantic_similarity(text_en, text_ms):
    embeddings_en = model.encode(text_en, convert_to_tensor=True)
    embeddings_ms = model.encode(text_ms, convert_to_tensor=True)
    cosine_score = util.cos_sim(embeddings_en, embeddings_ms).item()
    return cosine_score

# Example
original_en = "I love to eat apple and banana."
translated_ms = "Saya suka makan epal dan pisang." # Good translation
bad_translated_ms = "Saya suka makan ikan dan daging." # Bad translation

print(f"Good translation similarity: {get_semantic_similarity(original_en, translated_ms)}")
print(f"Bad translation similarity: {get_semantic_similarity(original_en, bad_translated_ms)}")